In [ ]:
import os
import sys
sys.path.append('../../../')
import pandas as pd
import datetime
from multiprocessing import Pool

In [ ]:
import plot_workflow.plotly_option as pltl_opt
import plot_workflow.plotly_workflow as pltl_wf

from preproc_p import workflow_cs_data
from preproc_p import workflow_chess_data
from preproc_p import preproc_tool
from preproc_p import workflow_calc_data
from preproc_p import workflow_tr_data
from preproc_p import filtration
from proc_p import processor as proc
from proc_p import workflow_input_data as workflow_input_data


from ml import calibr_restore as calibr_restore
from postproc_p import result_and_metrics as result_and_metrics

In [ ]:
well_name = '1976'
chess_file_name = 'Скв. 1976 (01.06.2019-30.11.2019).xls'
tr_name = "ТР Вынгаяхинское Январь 2020.csv"
# TODO убрать флаги
read_initial_data = True
plot_initial_data = True
create_input_data = True
auto_open_html = True
multiprocessing_on = True #TODO может быть убрать флаги, как лучше?

In [ ]:
static_data_full_path = preproc_tool.find_full_path_by_pattern(os.getcwd(), "*static_data.xlsx*")[0]

In [ ]:
current_path = os.getcwd()
time_mark = '' #datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)
dynamic_data_full_path = path_to_save + well_name + "_first_edit.csv"

In [ ]:
#left_boundary = [datetime.datetime(2018,8,1), datetime.datetime(2018,11,29)]
#right_boundary = [datetime.datetime(2018,11,5), datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2018,8,3), datetime.datetime(2018,11,29),   datetime.datetime(2019,2,19)]
#right_boundary = [datetime.datetime(2018,11,6), datetime.datetime(2019,2,4),  datetime.datetime(2019,2,28)]
#left_boundary = [datetime.datetime(2019,1,30)]
#right_boundary = [datetime.datetime(2019,2,28)]
left_boundary = [datetime.datetime(2018,6,27)]
right_boundary = [datetime.datetime(2021,12,9)]

In [ ]:
global_names = preproc_tool.GlobalNames()

In [ ]:
file_name_pattern = '*adaptation_//1976_calc_and_input.csv*'

In [ ]:
file_name = preproc_tool.find_full_path_by_pattern(os.getcwd(), file_name_pattern)[0]

In [ ]:
this_file = pd.read_csv(file_name, index_col = 'Время', parse_dates = True)
this_file.head()

In [ ]:
needed_data_time = datetime.datetime(2019,6,27,9)

In [ ]:
df_with_needed_data_time = this_file[this_file.index == needed_data_time]
row_in_prepared_data =  df_with_needed_data_time.iloc[0]
row_in_prepared_data[global_names.c_calibr_head_d + ' (ADAPT)']

In [ ]:
static_data = workflow_tr_data.Static_data()
static_data.h_pump_m = -1
this_state = workflow_input_data.all_ESP_data(static_data)
this_state = workflow_input_data.transfer_data_from_row_to_state(this_state, row_in_prepared_data, False)
this_state.__dict__

In [ ]:
dynamic_data_df = pd.DataFrame({'Время': [needed_data_time],
                                'p_intake_data_atm': [this_state.p_intake_data_atm],
                               'p_buf_data_atm': [this_state.p_buf_data_atm],
                               'tsep_c': [this_state.tsep_c],
                                
                               'cos_phi_data_d': [this_state.cos_phi_data_d],
                               'u_motor_data_v': [this_state.u_motor_data_v],
                               'active_power_cs_data_kwt': [this_state.active_power_cs_data_kwt],
                                'ESP_freq': [this_state.ESP_freq],

                               'qliq_m3day': [this_state.qliq_m3day],
                               'watercut_perc': [this_state.watercut_perc],
                               'rp_m3m3': [this_state.rp_m3m3],
                               
                                'd_choke_mm': [this_state.d_choke_mm], 
                                   
                               global_names.c_calibr_head_d: [row_in_prepared_data[global_names.c_calibr_head_d + ' (ADAPT)']],
                               global_names.c_calibr_power_d: [row_in_prepared_data[global_names.c_calibr_power_d + ' (ADAPT)']]})

In [ ]:
dynamic_data_df = dynamic_data_df.T
dynamic_data_df.columns = ['Параметр']
dynamic_data_df.to_excel('data_to_manual_vba.xlsx')
dynamic_data_df

In [ ]:
import scipy 

In [ ]:
preproc_tool.find_full_path_by_pattern(os.getcwd(), "*second_edit.csv*")

In [ ]:
file = pd.read_csv(preproc_tool.find_full_path_by_pattern(os.getcwd(), "*second_edit.csv*")[0], index_col = 'Время', parse_dates = True)

In [ ]:
file[global_names.active_power_kwt].dropna().plot()

In [ ]:
real = file[global_names.p_intake_atm].dropna()

In [ ]:
median = scipy.signal.medfilt(file[global_names.p_intake_atm].dropna().values, 53)

In [ ]:
df = pd.DataFrame({'active_power_real': real,
                  'active_power_median': median})

In [ ]:
df.plot()